#### Your `subjects.csv` should look like this:

```csv
MR ID,Subject_ID,subject,session
XNAT123,XY25_001,001,01
XNAT456,XY25_002,002,01
```

#### Running this Notebook is equivalent to execute below commands in Terminal. Choose what suits You more

Running `xnat-dcm2bids` for each row in a CSV file:

```bash
awk -F',' '{print $1, $3, $4}' subjects.csv | while read id sub ses; do \
  xnat-dcm2bids --bids-dir bids-dir --config ./config.json "$id" "$sub" "$ses"; \
done
```

where column mapping is as below (may be customized)

* first column `$1 = MR ID` → used as `id`
* third column `$3 = subject` → used as `sub`
* fourth column `$4 = session` → used as `ses`

Tip: You may skip "session" if all Your subjects have the same (ses-01)


In [1]:
#install dependencies:
#!pip install https://github.com/nencki-lobi/xnat_dcm2bids/archive/refs/heads/master.zip
#!git clone https://github.com/nencki-lobi/lobi-mri-scripts.git ~/lobi-mri-scripts

In [6]:
#define syntax of Your command

import pandas as pd

def generate_commands(csv_path, command_type, output_file="commands.txt"):
#function used to create command templates
    df = pd.read_csv(csv_path, dtype=str)
    commands = []

    for _, row in df.iterrows():
        bids_code = row['subject']
        xnat_id = row['MR ID']

        #here define Your commands
        if command_type == "mriqc":
            cmd = f"~/lobi-mri-scripts/run_mriqc.sh {bids_code} ~/Data/bids-dir ~/Data/bids-dir/derivatives/mriqc"
        elif command_type == "xnat-dcm2bids":
            cmd = f"xnat-dcm2bids {xnat_id} {bids_code} 1"
        if command_type.startswith("fmriprep"):
            cmd = f"~/lobi-mri-scripts/run_{command_type}.sh {bids_code} ~/Data/bids-dir ~/Data/bids-dir/derivatives/fmriprep"
        else:
            raise ValueError("Unknown command_type. Check commands' definitions")

        # Ensure environment modules (like ml mriqc) are loaded
        wrapped_command = (
            f'echo "Running {bids_code}... see logs/{bids_code}.log" && '
            f'bash -l -c "{cmd}" > logs/{bids_code}.log 2>&1'
        )
        commands.append(wrapped_command)

    # Save commands to file
    with open(output_file, "w") as f:
        f.write("\n".join(commands))



In [8]:
#create batch file

generate_commands("subjects.csv", command_type="fmriprep_kids_alicja")

In [4]:
#execute and save output to ./logs

!mkdir -p logs
!parallel -j 4 < commands.txt
#or without parallelization
#!bash commands.txt 

^C
